<a href="https://colab.research.google.com/github/bosinupebi/Data-Science/blob/master/index_from_website_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook makes a question answering chain with a specified website as a context data.

# Setting up

Install dependencies

In [ ]:
!pip3 install langchain==0.0.189
!pip3 install pinecone-client
!pip3 install openai
!pip3 install tiktoken
!pip3 install nest_asyncio


In [ ]:
!pip3 install --upgrade pinecone-client

Set up OpenAI API key

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

Set up Pinecone API keys

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="")

In [ ]:
pc.create_index(
    name="quickstart",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(
        cloud='aws', 
        region='us-west-2'
    ) 
) 

In [ ]:
import nest_asyncio
nest_asyncio.apply()



from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("")
pages = loader.load_and_split()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 200,
)

docs_chunks = text_splitter.split_documents(pages)

print(docs_chunks)

In [ ]:
import random
import string
import openai

#create open ai embedding then upsert vector
index = pc.Index("quickstart")

# Function to generate a random ID
def generate_random_id():
    return ''.join(random.choices(string.ascii_letters + string.digits, k=10))

# Process each document and upsert
for doc in docs_chunks:
    # Clean and prepare page content
    cleaned_content = ' '.join(doc.page_content.replace('\n', ' ').split())

    # Create embedding
    response = openai.Embedding.create(
        input=cleaned_content,
        model='text-embedding-3-small'
    )
    embeds = [record['embedding'] for record in response['data']]

    # Generate a random ID for the document
    doc_id = generate_random_id()

    # Prepare metadata
    doc_metadata = {
        "source": doc.metadata['source'],
        "page": doc.metadata['page'],
        "text": cleaned_content
    }

    # Upsert to Pinecone
    index.upsert(
        vectors=[{
            "id": doc_id,
            "values": embeds[0],  # Assuming one embedding per document
            "metadata": doc_metadata
        }]
    )